base de dados:
- dados dos posts wordpress (pre-migração)
- dados dos posts wix (pós-migração)

Como organizar os dados de forma que facilite a migração e tratamento entre as duas plataformas?

criar um arquivo 